In [163]:
import numpy as np
import pandas as pd
import random
from pprint import pprint ########## for nice printing of the branches of the tree

# For reading of iris dataset

In [164]:
df = pd.read_csv("C:\\Users\\gp\\Desktop\\Camera\\Iris.csv",delimiter=",")
df = df.drop("Id", axis=1)##### for the removal of unneccessary id column in datasets
df
df.head()


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [165]:
len(df)

150

In [166]:
df.info() #####for checking for any any null value presence

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
species         150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


# my own train and split function

In [168]:
#####from sklearn.model_selection import train_test_split
#####3X_train,X_test,Y_train,Y_test=train_test_split(df,y,random_state=0)######### splitting our data into training and testing

In [169]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [180]:
random.seed(0)
train_df, test_df = train_test_split(df, test_size=20)

# for checking purity of data

In [181]:
def check_purity(data):
    
    label_column = data[:, -1]              ##########taking whole data
    unique_classes = np.unique(label_column)####### we are going to calculate unique values

    if len(unique_classes) == 1:
        return True
    else:
        return False
    ###########if iris setosa or other class is present only return true


# for classification of data while checking purity of data

In [182]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

# FOR POTENTIAL SPLITS IN DATA because every taking avg between two consecutive points

In [183]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):        # excluding the last column which is the label
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)           #####calcualting unique values and next step we are getting potential split

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2   ###### as said by sir taking in between of 1.8 and 2.1
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

# for every split in a function(note it is not a best caser split,it is just splitting data for us)

In [184]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    
    return data_below, data_above

# for calculating the entropy two below fuction are used

In [175]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)######### It is going to return as both uniques values as well as there count

    probabilities = counts / counts.sum()##### note my friend probailities is actually a numpy array
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [176]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [177]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999            ############ we are taking here maximum value for us
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [178]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS    ######## without its global keyword it will give error (due to recursion process)
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):########we can define max_depth and min_samples by ourselves
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # instantiate sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [179]:
tree = decision_tree_algorithm(train_df, max_depth=3)
pprint(tree)

{'petal_length <= 2.5999999999999996': [{'sepal_width <= 3.3499999999999996': [0.2,
                                                                               {'sepal_width <= 3.8499999999999996': [0.2,
                                                                                                                      0.4]}]},
                                        {'petal_length <= 4.75': [{'petal_length <= 4.15': [1.0,
                                                                                            1.5]},
                                                                  {'petal_length <= 5.35': [1.8,
                                                                                            2.1]}]}]}
